# Predicting the past, United States

In [3]:
import sys
sys.path.append('analysis/')

import analysis as an
import import_data as imd

## Abstract
In an effort to produce a cross comparison data of the US fleet, _CYCLUS_, an agent-based simulation software was used to simulate and analyze all reactors that have been deployed from 1965. US fleet of nuclear reactors were obtained from a text file and used to specify the reactor name, type, core mass, capacity, deployment date, and closure date.

To be continued

## Methodology
This project relies on the following: recipes for reactor fuel, information regarding reactors, choosing the reactors to deploy, running the simulation, and analyzing the simulation output. 

### Obtaining Recipes for Simulation
#### Files Used

##### Input
*  [uox.csv](import_data/vision_recipes/uox.csv) - contains isotopes and compositions of UOX, and SNF at different burnups
*  [recipe_template.xml](templates/recipe_template.xml) - an xml template that provides the structure of the recipe file that complies with _CYCLUS_

##### Script
*  [import_data.py](import_data.py) - a script that is used to import, process and produce recipes for UOX, and SNF

##### Output
*  [uox_51.xml](cyclus/input/recipes/uox_51/xml) - an xml file containing the isotopes and their compositions


Recipes that describe the composition of spent nuclear fuel for uranium oxide (UOX) at the burn up of 51 GWd/MTHM, was obtained from `uox.csv`, a text file that contains the results of origen simulations. `import_data` script was used to obtain the necessary isotope and composition data.

This involved importing the text file (`uox.csv`), and a template for recipes (`recipe_template.xml`), extracting the relevant isotope, and composition information for the specified burnup, and using `jinja2` to render the imported template.

### Obtaining Reactors for Simulation
#### Files Used

##### Input
*  [US_Fleet.txt](import_data/fleetcomp/US_Fleet.txt) - fleetcomp data of US nuclear power plants
*  [reactors_template.xml](templates/reactors_template.xml) - an xml template that provides the structure of the recipe file that complies with _CYCAMORE_ reactors input file

##### Script
*  [import_data.py](import_data.py) - a script that is used to import, process and produce recipes for UOX, and SNF

##### Output
*  [112 Reactor files](cyclus/input/reactors) - 112 files containing information regarding a reactor deployed in the United States since 1965

### Adding Deployment Data
#### Files Used

##### Input
*  [US_Fleet.txt](import_data/fleetcomp/US_Fleet.txt) - fleetcomp data of US nuclear power plants
*  [reactors_template.xml](templates/reactors_template.xml) - an xml template that provides the structure of the recipe file that complies with _CYCAMORE_ reactors input file

##### Script
*  [import_data.py](import_data.py) - a script that is used to import, process and produce recipes for UOX, and SNF

##### Output
*  [112 Reactor files](cyclus/input/reactors) - 112 files containing information regarding a reactor deployed in the United States since 1965

### Obtaining Reactors for Simulation
#### Files Used

##### Input
*  [US_Fleet.txt](import_data/fleetcomp/US_Fleet.txt) - fleetcomp data of US nuclear power plants
*  [reactors_template.xml](templates/reactors_template.xml) - an xml template that provides the structure of the recipe file that complies with _CYCAMORE_ reactors input file

##### Script
*  [import_data.py](import_data.py) - a script that is used to import, process and produce recipes for UOX, and SNF

##### Output
*  [112 Reactor files](cyclus/input/reactors) - 112 files containing information regarding a reactor deployed in the United States since 1965


## Results

## Discussion

## Future Goals
+  Specify actual burnup for reactors
    - Currently, the reactors deployed in Cyclus are all deployed with a burnup of 51 GWd/MTHM. 
    
+  Specify reactor n_assembly_core and batch
    - Reactors were deployed in cyclus without the actual number of assemblies per core and batch. The typical values were used for each reactor type (Source needs to be added for the typical values). While the effects of changing these values may not be significant in the outcome of the simulation, specifying such information may benefit the accuracy of the simulation results.
    
+  Apply a similar analysis to different regions such as the EU or in the world


## References
*  US Nuclear Generating Statistics. (n.d.). Retrieved from https://www.nei.org/Knowledge-Center/Nuclear-Statistics/US-Nuclear-Power-Plants/US-Nuclear-Generating-Statistics

*  US Nuclear Refueling Outage Days. (n.d.). Retrieved from https://www.nei.org/Knowledge-Center/Nuclear-Statistics/US-Nuclear-Power-Plants/US-Nuclear-Refueling-Outage-Days

*  EIA. (n.d.). Capacity factor of nuclear power plants in the U.S. from 1975 to 2016*. In Statista - The Statistics Portal. Retrieved from https://www.statista.com/statistics/191201/capacity-factor-of-nuclear-power-plants-in-the-us-since-1975/. 